In [ ]:
from hdx.utilities.easy_logging import setup_logging
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
import os
import numpy as np
import pandas as pd

In [ ]:
setup_logging()
Configuration.create(hdx_site='prod', user_agent='A_Quick_Example', hdx_read_only=True)

In [ ]:
#Check if the dataset has at least 1 resource of the required file type(s).

def check_type(dataset, file_types=[]):
    temp_dataset = Dataset.read_from_hdx(dataset)
    temp_dataset.separate_resources()
    if (len(temp_dataset.resources) > 0):
        if (len(file_types) > 0):
            if (not set(temp_dataset.get_filetypes()).isdisjoint(file_types)): 
                    return True
        else :
            return True
    return False

In [ ]:
# Check if the dataset is tagged with HXL tag, not provided by HXL

def check_organization(dataset):
    if dataset.get_organization()['title'] != 'Humanitarian Exchange Language(HXL)':
            return True
    return False

In [ ]:
# Download one dataset with certain type(s), read it into Dataframe, 
# add all headers, tags and dataset names to our DataFrame,
# and delete the dataset

def process_dataset_2(dataset, file_type, dataframe, download_path, index, row_limit = 10):
    global count
# Download one dataset and read it into a DataFrame 
    if (file_type == None):
        url, path = dataset_1.resources[0].download(download_path)
        pandas_dataset = pd.read_csv(path)
    else:
        if (file_type not in dataset.get_filetypes()):
            return 'Error: Required file type not in dataset OR dataset does not contain any resources.'
        try:
            url, path = dataset.resources[dataset.get_filetypes().index(file_type)].download(download_path)
            print('Resource URL %s downloaded to %s' % (url, path))
            pandas_dataset = pd.read_csv(path, encoding='latin-1')
            pandas_dataset = pandas_dataset.head(row_limit)
        except:
            return 'Unknown error.'
     
    #if "HXL" in os.path.basename(path) or "hxl" in os.path.basename(path):
        #return dataset_df
    
# Add headers, tags and data to our DataFrame if current dataset not empty
        if (not pandas_dataset.empty):
            dataset_df = pandas_dataset
            headers = list(dataset_df.columns.values)
            tags = list(dataset_df.iloc[0,:])
            for i in range(len(headers)):
                try:
                    dic = {'Header': headers[i], 'Tag': tags[i], 'Data': list(dataset_df.iloc[1:, i]), 
                           'Relative Column Position': (i+1) / len(dataset_df.columns), 
                           'Dataset_name': os.path.basename(path), 'Index': index}
                    headers_and_tags.loc[len(headers_and_tags)] = dic
                except:
                    print("Error: different number of headers and tags")
            count += 1
        os.remove(path)
        print("File Removed!")
        return

In [ ]:
# Search for all datasets with HXL tags

datasets_HXL = Dataset.search_in_hdx('HXL')
len(datasets_HXL)

In [ ]:
# Create a DataFrame for all headers and tags

col_names = ['Header', 'Tag', 'Data','Relative Column Position','Dataset_name', 'Index']
headers_and_tags= pd.DataFrame(columns = col_names)

In [ ]:
count = 0
for i in range(300):
    process_dataset_2(datasets_HXL[i], 'CSV', headers_and_tags, 'HDX File Holder', count)
    print(i)

In [ ]:
headers_and_tags.to_csv("headerandtag.csv")

In [ ]:
from ast import literal_eval
headers_and_tags_f = pd.read_csv('headerandtag.csv')
new_data = headers_and_tags_f['Data'].replace("'",'',regex=True).replace('\[','',regex=True).replace('\]','',regex=True).str.split(",")
headers_and_tags_f['Data'] = new_data


In [ ]:
##Splits HXL Tags into Tags and additional attributes
def splitter_tag(string):
    return string.split('+')

In [ ]:
tag_column = headers_and_tags_f['Tag']
stringed = tag_column.apply(str)
splitted_tag = stringed.apply(splitter_tag)
tags_only = [i[0] for i in splitted_tag]

In [ ]:
headers_and_tags_f['Tag']= tags_only

In [ ]:
#Gives a count in descending order of only the HXL Tags in the Dataset
hxl_tag_count = headers_and_tags_f.drop(columns=['Unnamed: 0','Data','Relative Column Position','Dataset_name','Index']).groupby(['Tag']).count().sort_values(by='Header',ascending=False)
#hxl_tag_count

In [ ]:
#Groups the Headers by the Tags they are associated with
headers_by_tag = headers_and_tags_f.groupby(['Tag'])['Header'].apply(lambda headers: ','.join(headers))


In [ ]:
#cash = headers_and_tags_f.drop(columns=['Unnamed: 0','Header','Relative Column Position','Dataset_name','Index']).groupby(['Tag'])


In [ ]:
import itertools

data_by_tag = headers_and_tags_f.groupby(['Tag'])['Data'].apply(list)
data_by_tag = pd.DataFrame(data_by_tag)
data_by_tag = data_by_tag['Data'].apply(lambda x:list(itertools.chain.from_iterable(x)))
data_by_tag.to_csv('data_by_tag.csv')